# Common imports & library functions

In [1]:
import collections
from collections import defaultdict, Counter
from dataclasses import dataclass, field
import doctest
import functools
import itertools
from itertools import count
import math
import re
from copy import deepcopy
from enum import Enum

from utils import product, neighboring_cells

# Day 16: Ticket Translation

In [1]:
def parse_ticket(line):
    """
    >>> parse_ticket('1,5,99')
    [1, 5, 99]
    """
    return [int(i) for i in line.strip().split(',')]

def parse_ranges(line):
    """
    >>> name, values = parse_ranges('class: 50-55 or 587-590')
    >>> name
    'class'
    >>> list(sorted(values))
    [50, 51, 52, 53, 54, 55, 587, 588, 589, 590]
    """
    name, ranges = line.split(': ')
    ranges = ranges.split(' or ')
    values = set()
    for lo_hi in ranges:
        lo, hi = lo_hi.split('-')
        values.update(range(int(lo), int(hi) + 1))
    return name, values

def parse_input(input):
    valid_ranges = {}
    your_ticket = None
    other_tickets = []
    for line in input:
        line = line.strip()
        if ': ' in line:
            name, values = parse_ranges(line)
            valid_ranges[name] = values
        elif ',' in line:
            ticket = parse_ticket(line)
            if your_ticket is None:
                your_ticket = ticket
            else:
                other_tickets.append(ticket)
    return valid_ranges, your_ticket, other_tickets

def invalid_ticket_values(valid_ranges, tickets):
    valid_values = functools.reduce(set.union, valid_ranges.values(), set())
    for ticket_value in itertools.chain(*tickets):
        if ticket_value not in valid_values:
            yield ticket_value

In [45]:
doctest.run_docstring_examples(parse_ticket, globs=None, verbose=True)
doctest.run_docstring_examples(parse_ranges, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_ticket('1,5,99')
Expecting:
    [1, 5, 99]
ok
Finding tests in NoName
Trying:
    name, values = parse_ranges('class: 50-55 or 587-590')
Expecting nothing
ok
Trying:
    name
Expecting:
    'class'
ok
Trying:
    list(sorted(values))
Expecting:
    [50, 51, 52, 53, 54, 55, 587, 588, 589, 590]
ok


In [46]:
notes = """
    class: 1-3 or 5-7
    row: 6-11 or 33-44
    seat: 13-40 or 45-50

    your ticket:
    7,1,14

    nearby tickets:
    7,3,47
    40,4,50
    55,2,20
    38,6,12
""".splitlines()

valid_ranges, your_ticket, other_tickets = parse_input(notes)
assert sum(invalid_ticket_values(valid_ranges, other_tickets)) == 71

In [85]:
def valid_tickets(valid_ranges, tickets):
    valid_values = functools.reduce(set.union, valid_ranges.values(), set())
    for ticket in tickets:
        if all(v in valid_values for v in ticket):
            yield ticket

def guess_field(valid_ranges, values):
    for name, valid in valid_ranges.items():
        if all(v in valid for v in values):
            yield name

def first(s):
    return list(s)[0]

def guess_fields(valid_ranges, your_ticket, other_tickets):
    # Step 1: Come up with possible guesses based on which fields' valid ranges are
    # compatible with the values encountered in tickets.
    guesses = []
    tickets = valid_tickets(valid_ranges, other_tickets + [your_ticket])
    for values in zip(*tickets):
        guesses.append(set(guess_field(valid_ranges, values)))

    # Step 2: Prune guesses until there's only one choice for each field, or a contradiction 
    # is hit (an empty set is left).
    while True:
        updated = False
        
        for field_i, choices_i in enumerate(guesses):
            if len(choices_i) == 0:
                raise Exception(f'No guesses remaining for field {field_i}')
            elif len(choices_i) == 1:
                guess = first(choices_i)
                for field_j, choices_j in enumerate(guesses):
                    if field_i == field_j:
                        continue
                    elif guess in choices_j:
                        choices_j.remove(guess)
                        updated = True
            
        if not updated:
            return [first(choices) for choices in guesses]


In [89]:
notes = """
    class: 0-1 or 4-19
    row: 0-5 or 8-19
    seat: 0-13 or 16-19

    your ticket:
    11,12,13

    nearby tickets:
    3,9,18
    15,1,5
    5,14,9
""".splitlines()

valid_ranges, your_ticket, other_tickets = parse_input(notes)
assert guess_fields(valid_ranges, your_ticket, other_tickets) == ['row', 'class', 'seat']

In [93]:
%%time
# Final answers
with open('day16.txt') as f:
    notes = f.readlines()
    valid_ranges, your_ticket, other_tickets = parse_input(notes)
    print('Part 1: ', sum(invalid_ticket_values(valid_ranges, other_tickets)))
    
    fields = guess_fields(valid_ranges, your_ticket, other_tickets)
    print('Part 2: ', product(value for field, value in zip(fields, your_ticket) if field.startswith('departure')))

Part 1:  32842
Part 2:  2628667251989
Wall time: 13 ms


# Day 17: Conway Cubes

In [159]:
ds = (-1, 0, 1)

class CubeState(Enum):
    INACTIVE = 0
    ACTIVE = 1

CubeGrid = lambda init={}: defaultdict(lambda: CubeState.INACTIVE, init)

def neighboring_cells(cell):
    """
    >>> list(neighboring_cells((0, 0)))
    [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    >>> len(list(neighboring_cells((0, 0, 0))))
    26
    >>> len(list(neighboring_cells((-1, 0, 0, 1))))
    80
    """
    for deltas in itertools.product(*[ds]*len(cell)):
        if all(d == 0 for d in deltas):
            continue
        yield tuple(c+d for c, d in zip(cell, deltas))

def parse_state(state_2d, dims=3):
    rows = state_2d.strip().splitlines()
    state_3d = CubeGrid()
    for y, row in enumerate(rows):
        for x, state in enumerate(row.strip()):
            coord = (x, y) + (0,) * (dims - 2)
            state_3d[coord] = CubeState.INACTIVE if state == '.' else CubeState.ACTIVE
    return state_3d

def num_active(state):
    return sum(cell_state.value for cell_state in state.values())

def step(initial_state, n_steps=1):
    """
    >>> initial_state = parse_state('''
    ...    .#.
    ...    ..#
    ...    ###
    ... ''')
    >>> num_active(step(initial_state, n_steps=6))
    112
    """
    prev_state = CubeGrid(initial_state)
    
    while n_steps > 0:
        # Visit cells and their neighbors this iteration, extending the boundary
        # of visited cells by one.
        to_visit = set(prev_state.keys())
        for cell in prev_state:
            to_visit.update(neighboring_cells(cell))
        #print('Steps remaining', n_steps, ', cells to visit', len(to_visit))

        # Perform a single update step.
        next_state = CubeGrid()
        for cell in to_visit:
            state = prev_state[cell]
            # Count the number of living neighbors and update current state.
            live_neighbors = sum(prev_state[n].value for n in neighboring_cells(cell))
            if state == CubeState.ACTIVE and live_neighbors not in (2, 3):
                next_state[cell] = CubeState.INACTIVE
            elif state == CubeState.INACTIVE and live_neighbors == 3:
                next_state[cell] = CubeState.ACTIVE
            else:
                next_state[cell] = state

        prev_state = next_state
        n_steps -= 1
    
    return prev_state



In [160]:
doctest.run_docstring_examples(neighboring_cells, globs=None, verbose=True)
doctest.run_docstring_examples(step, globs=None, verbose=True)

Finding tests in NoName
Trying:
    list(neighboring_cells((0, 0)))
Expecting:
    [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
ok
Trying:
    len(list(neighboring_cells((0, 0, 0))))
Expecting:
    26
ok
Trying:
    len(list(neighboring_cells((-1, 0, 0, 1))))
Expecting:
    80
ok
Finding tests in NoName
Trying:
    initial_state = parse_state('''
       .#.
       ..#
       ###
    ''')
Expecting nothing
ok
Trying:
    num_active(step(initial_state, n_steps=6))
Expecting:
    112
ok


In [161]:
%%time
# Final answers
with open('day17.txt') as f:
    initial_state_2d = f.read()
    initial_state_3d = parse_state(initial_state_2d, dims=3)
    print('Part 1: ', num_active(step(initial_state_3d, 6)))
    
    initial_state_4d = parse_state(initial_state_2d, dims=4)
    print('Part 2: ', num_active(step(initial_state_4d, 6)))

Part 1:  306
Part 2:  2572
Wall time: 47.8 s


# Day 18: Operation Order

In [4]:
OPS = {
    '*': lambda a, b: a * b,
    '+': lambda a, b: a + b
}

def tokenize(expr):
    """
    >>> tokenize('1 + 2 * 3 + 4 * 5 + 6')
    ['1', '+', '2', '*', '3', '+', '4', '*', '5', '+', '6']
    >>> tokenize('1 + (2 * 3) + (4 * (5 + 6))')
    ['1', '+', '(', '2', '*', '3', ')', '+', '(', '4', '*', '(', '5', '+', '6', ')', ')']
    """
    return expr.replace('(', '( ').replace(')', ' )').split(' ')

def parse_next(tokens, idx, stack):
    if idx >= len(tokens):
        return idx
    token = tokens[idx]
    if token == '(':
        idx = idx + 1
        while True:
            idx = parse_next(tokens, idx, stack)
            if stack[-1] == ')':
                stack.pop()
                return idx
    elif token == ')':
        stack.append(token)
        return idx + 1
    elif token in ('*', '+'):
        idx = parse_next(tokens, idx + 1, stack)
        b, a = stack.pop(), stack.pop()
        stack.append([token, a, b])
        return idx
    else:
        stack.append(int(token))
        return idx + 1

def parse(tokens):
    """
    >>> parse(tokenize('1 * 2 + 3'))
    ['+', ['*', 1, 2], 3]
    >>> parse(tokenize('(2 * 3) + (1 * (4 + 5))'))
    ['+', ['*', 2, 3], ['*', 1, ['+', 4, 5]]]
    """
    idx = 0
    stack = []
    while idx < len(tokens):
        idx = parse_next(tokens, idx, stack)
    return stack.pop()

def evaluate(parsed):
    """
    >>> evaluate(parse(tokenize('1 * 2 + 3')))
    5
    >>> evaluate(parse(tokenize('(2 * 3) + (1 * (4 + 5))')))
    15
    """
    if isinstance(parsed, int): return parsed
    op, a, b = parsed[0], evaluate(parsed[1]), evaluate(parsed[2])
    return OPS[op](a, b)

def parse_and_evaluate(expr):
    return evaluate(parse(tokenize(expr.strip())))

In [3]:
doctest.run_docstring_examples(tokenize, globs=None, verbose=True)
doctest.run_docstring_examples(parse, globs=None, verbose=True)
doctest.run_docstring_examples(evaluate, globs=None, verbose=True)

Finding tests in NoName
Trying:
    tokenize('1 + 2 * 3 + 4 * 5 + 6')
Expecting:
    ['1', '+', '2', '*', '3', '+', '4', '*', '5', '+', '6']
ok
Trying:
    tokenize('1 + (2 * 3) + (4 * (5 + 6))')
Expecting:
    ['1', '+', '(', '2', '*', '3', ')', '+', '(', '4', '*', '(', '5', '+', '6', ')', ')']
ok
Finding tests in NoName
Trying:
    parse(tokenize('1 * 2 + 3'))
Expecting:
    ['+', ['*', 1, 2], 3]
ok
Trying:
    parse(tokenize('(2 * 3) + (1 * (4 + 5))'))
Expecting:
    ['+', ['*', 2, 3], ['*', 1, ['+', 4, 5]]]
ok
Finding tests in NoName
Trying:
    evaluate(parse(tokenize('1 * 2 + 3')))
Expecting:
    5
ok
Trying:
    evaluate(parse(tokenize('(2 * 3) + (1 * (4 + 5))')))
Expecting:
    15
ok


In [84]:
def end_of(tokens, idx):
    stack = [tokens[idx]]
    for idx in range(idx + 1, len(tokens)):
        if tokens[idx] == '(':
            stack.append(tokens[idx])
        elif tokens[idx] == ')':
            stack.pop()
        if not stack:
            return idx

    
def parse_groups(tokens):
    terms = []
    idx = 0
    while idx < len(tokens):
        token = tokens[idx]
        if token == '(':
            end = end_of(tokens, idx)
            group_tokens = tokens[idx+1:end]
            terms.append(parse_groups(group_tokens))
            idx = end
        elif token in OPS:
            terms.append(token)
        elif token != ')':
            terms.append(int(token))
        idx += 1
    return terms


def evaluate_op(terms, op):
    # Return early if it's an operator or number.
    if isinstance(terms, str) or isinstance(terms, int):
        return terms
    # Otherwise, it's a complex expression: simplify all terms.
    terms = [evaluate_op(term, op) for term in terms]
    idx = 0
    result = []
    while idx < len(terms):
        term = terms[idx]
        if term == op:
            lhs = result.pop()
            rhs = terms[idx+1]
            # Evaluate it on the spot.
            result.append(OPS[op](lhs, rhs))
            idx += 2
        else:
            result.append(term)
            idx += 1
    return result[0] if len(result) == 1 else result


def evaluate_advanced(terms, precedence='+*'):
    terms = [evaluate_advanced(term, precedence) if isinstance(term, list) else term
             for term in terms]
    for op in precedence:
        terms = evaluate_op(terms, op)
    return terms

def parse_and_evaluate_advanced(expr):
    """
    >>> parse_and_evaluate_advanced('1 + 2 * 3 + 4 * 5 + 6')
    231
    >>> parse_and_evaluate_advanced('2 * 3 + (4 * 5)')
    46
    >>> parse_and_evaluate_advanced('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))')
    669060
    >>> parse_and_evaluate_advanced('1 + (2 * 3) + (4 * (5 + 6))')
    51
    >>> parse_and_evaluate_advanced('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2')
    23340
    """
    return evaluate_advanced(parse_groups(tokenize(expr.strip())))

In [86]:
doctest.run_docstring_examples(parse_and_evaluate_advanced, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_and_evaluate_advanced('1 + 2 * 3 + 4 * 5 + 6')
Expecting:
    231
ok
Trying:
    parse_and_evaluate_advanced('2 * 3 + (4 * 5)')
Expecting:
    46
ok
Trying:
    parse_and_evaluate_advanced('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))')
Expecting:
    669060
ok
Trying:
    parse_and_evaluate_advanced('1 + (2 * 3) + (4 * (5 + 6))')
Expecting:
    51
ok
Trying:
    parse_and_evaluate_advanced('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2')
Expecting:
    23340
ok


In [87]:
%%time
# Final answers
with open('day18.txt') as f:
    exprs = [e.strip() for e in f]
    print('Part 1: ', sum(parse_and_evaluate(e) for e in exprs))
    print('Part 2: ', sum(parse_and_evaluate_advanced(e) for e in exprs))

Part 1:  5783053349377
Part 2:  74821486966872
Wall time: 42 ms


# Day 19: Monster Messages

In [298]:
# A cheat because it supports recursive (i.e. not-so-regular) regexes :(
import regex

def compile_grammar(rules, special_cases=False):
    def rule_8():
        rule_42 = compile_rule(42)
        return rf'({rule_42}+)'

    def rule_11():
        rule_42 = compile_rule(42)
        rule_31 = compile_rule(31)
        return rf'(?<rule_11>{rule_42}(?&rule_11)*{rule_31})'

    @functools.lru_cache(maxsize=1024)
    def compile_rule(num):
        if special_cases and num == 8:
            return rule_8()
        elif special_cases and num == 11:
            return rule_11()
        rule = rules[num]
        if rule.startswith('"'):
            return rule[1:-1]
        
        disj_parts = []
        for disj in rule.split(' | '):
            conj_parts = []
            for conj in disj.split(' '):
                conj_parts.append(compile_rule(int(conj)))
            disj_parts.append('({})'.format(''.join(conj_parts)))
        return '({})'.format('|'.join(disj_parts))

    grammar = {}
    for num, rule in rules.items():
        grammar[num] = compile_rule(num)
    return grammar

def parse_grammar(lines, special_cases=False):
    rules = {}
    for line in lines:
        if ':' not in line: break
        num, rule = line.strip().split(': ')
        rules[int(num)] = rule
    return compile_grammar(rules, special_cases)

def parse_strings(lines):
    for line in lines:
        if ':' not in line:
            yield line.strip()

def matches_grammar(string, grammar, rule=0):
    pattern = regex.compile(grammar[rule])
    return pattern.fullmatch(string) is not None

def num_matching_strings(lines, special_cases=False):
    grammar = parse_grammar(lines, special_cases)
    strings = parse_strings(lines)
    return sum(matches_grammar(s, grammar) for s in strings)


In [299]:
test_input = """
    0: 4 1 5
    1: 2 3 | 3 2
    2: 4 4 | 5 5
    3: 4 5 | 5 4
    4: "a"
    5: "b"

    ababbb
    bababa
    abbbab
    aaabbb
    aaaabbb""".strip().splitlines()

assert num_matching_strings(test_input) == 2
parse_grammar(test_input)

{0: '((a((((aa)|(bb))((ab)|(ba)))|(((ab)|(ba))((aa)|(bb))))b))',
 1: '((((aa)|(bb))((ab)|(ba)))|(((ab)|(ba))((aa)|(bb))))',
 2: '((aa)|(bb))',
 3: '((ab)|(ba))',
 4: 'a',
 5: 'b'}

In [300]:
test_input = """
    42: 9 14 | 10 1
    9: 14 27 | 1 26
    10: 23 14 | 28 1
    1: "a"
    11: 42 31
    5: 1 14 | 15 1
    19: 14 1 | 14 14
    12: 24 14 | 19 1
    16: 15 1 | 14 14
    31: 14 17 | 1 13
    6: 14 14 | 1 14
    2: 1 24 | 14 4
    0: 8 11
    13: 14 3 | 1 12
    15: 1 | 14
    17: 14 2 | 1 7
    23: 25 1 | 22 14
    28: 16 1
    4: 1 1
    20: 14 14 | 1 15
    3: 5 14 | 16 1
    27: 1 6 | 14 18
    14: "b"
    21: 14 1 | 1 14
    25: 1 1 | 1 14
    22: 14 14
    8: 42
    26: 14 22 | 1 20
    18: 15 15
    7: 14 5 | 1 21
    24: 14 1

    abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
    bbabbbbaabaabba
    babbbbaabbbbbabbbbbbaabaaabaaa
    aaabbbbbbaaaabaababaabababbabaaabbababababaaa
    bbbbbbbaaaabbbbaaabbabaaa
    bbbababbbbaaaaaaaabbababaaababaabab
    ababaaaaaabaaab
    ababaaaaabbbaba
    baabbaaaabbaaaababbaababb
    abbbbabbbbaaaababbbbbbaaaababb
    aaaaabbaabaaaaababaa
    aaaabbaaaabbaaa
    aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
    babaaabbbaaabaababbaabababaaab
    aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
""".strip().splitlines()

assert num_matching_strings(test_input, special_cases=True) == 12
grammar = parse_grammar(test_input, special_cases=True)
grammar

{42: '((((b((a((bb)|(ab)))|(b((((a)|(b))((a)|(b)))))))|(a((b((bb)))|(a((bb)|(a((a)|(b))))))))b)|(((((((aa)|(ab))a)|(((bb))b))b)|(((((((a)|(b))a)|(bb))a))a))a))',
 9: '((b((a((bb)|(ab)))|(b((((a)|(b))((a)|(b)))))))|(a((b((bb)))|(a((bb)|(a((a)|(b))))))))',
 10: '((((((aa)|(ab))a)|(((bb))b))b)|(((((((a)|(b))a)|(bb))a))a))',
 1: 'a',
 11: '(?<rule_11>((((b((a((bb)|(ab)))|(b((((a)|(b))((a)|(b)))))))|(a((b((bb)))|(a((bb)|(a((a)|(b))))))))b)|(((((((aa)|(ab))a)|(((bb))b))b)|(((((((a)|(b))a)|(bb))a))a))a))(?&rule_11)*((b((b((a((ba)))|(b((aa)))))|(a((b((ab)|(((a)|(b))a)))|(a((ba)|(ab)))))))|(a((b((((ab)|(((a)|(b))a))b)|(((((a)|(b))a)|(bb))a)))|(a((((ba))b)|(((ba)|(bb))a)))))))',
 5: '((ab)|(((a)|(b))a))',
 19: '((ba)|(bb))',
 12: '((((ba))b)|(((ba)|(bb))a))',
 16: '((((a)|(b))a)|(bb))',
 31: '((b((b((a((ba)))|(b((aa)))))|(a((b((ab)|(((a)|(b))a)))|(a((ba)|(ab)))))))|(a((b((((ab)|(((a)|(b))a))b)|(((((a)|(b))a)|(bb))a)))|(a((((ba))b)|(((ba)|(bb))a))))))',
 6: '((bb)|(ab))',
 2: '((a((ba)))|(b((aa))

In [301]:
%%time
# Final answers
with open('day19.txt') as f:
    lines = [l.strip() for l in f]
    print('Part 1: ', num_matching_strings(lines))
    print('Part 2: ', num_matching_strings(lines, special_cases=True))

Part 1:  210
Part 2:  422
Wall time: 86 ms


# Day 20: Jurassic Jigsaw

In [407]:
from typing import List, Set
import heapq

@dataclass(frozen=True, eq=True)
class Tile:
    tile_id: int
    top: str
    bottom: str
    left: str
    right: str
    flipped: bool = False
    rotation: int = 0

    @staticmethod
    def from_record(record):
        tile_id, tile_map = record.split(':')
        tile_id = int(tile_id.strip().split()[-1])
        rows = tile_map.strip().splitlines()
        top = rows[0]
        bottom = rows[-1]
        left = ''.join(row[0] for row in rows)
        right = ''.join(row[-1] for row in rows)
        return Tile(tile_id, top, bottom, left, right)

    def edges(self):
        return set([self.top, self.bottom, self.left, self.right])

    def flip(self):
        return Tile(self.tile_id,
                    self.top[::-1], self.bottom[::-1], self.right, self.left,
                    not self.flipped, self.rotation)
    
    def rotate(self, amount):
        rotation = (self.rotation + amount) % 360
        if amount == 90:
            return Tile(self.tile_id, self.left[::-1], self.right[::-1], self.bottom, self.top, self.flipped, rotation)
        elif amount == 180:
            return Tile(self.tile_id, self.bottom[::-1], self.top[::-1], self.right[::-1], self.left[::-1], self.flipped, rotation)
        elif amount == 270:
            return Tile(self.tile_id, self.right, self.left, self.top[::-1], self.bottom[::-1], self.flipped, rotation)
        else:
            return self

    def orientations(self):
        for deg in [0, 90, 180, 270]:
            yield self.rotate(deg)
            yield self.flip().rotate(deg)

    def tile_key(self):
        return (self.tile_id, self.flipped, self.rotation)

    def fits_left_of(self, tile):
        return self.right == tile.left

    def fits_above(self, tile):
        return self.bottom == tile.top

    def __hash__(self):
        return hash(self.tile_key())


@dataclass(frozen=True, eq=True)
class Edge:
    position: str
    tile1: Tile
    tile2: Tile


@dataclass(eq=True, order=True)
class TileMap:
    tiles: Set[Tile] = field(default_factory=set)
    edges: Set[Edge] = field(default_factory=set)

    def try_placing(self, tile):
        if not self.tiles:
            yield TileMap(self.tiles | {tile}, self.edges)

        for other_tile in self.tiles:
            for oriented in tile.orientations():
                oriented_key = oriented.tile_key()
                if oriented.fits_left_of(other_tile):
                    yield self.place_left_of(oriented, other_tile)
                if other_tile.fits_left_of(oriented):
                    yield self.place_left_of(other_tile, oriented)
                if oriented.fits_above(other_tile):
                    yield self.place_above(oriented, other_tile)
                if other_tile.fits_above(oriented):
                    yield self.place_above(other_tile, oriented)
        
    def place_left_of(self, t1, t2):
        return TileMap(self.tiles | {t1, t2},
                       self.edges | {Edge('left', t1, t2)})

    def place_above(self, t1, t2):
        return TileMap(self.tiles | {t1, t2},
                       self.edges | {Edge('above', t1, t2)})

    def complete_edges(self):
        edge_completions = defaultdict(lambda: None)
        all_edges = set()
        
        def add_edge(edge):
            if edge in all_edges: return False
            all_edges.add(edge)
            edge_completions[(edge.tile1, edge.position)] = edge.tile2
            edge_completions[(edge.position, edge.tile2)] = edge.tile1
            return False

        for edge in self.edges:
            add_edge(edge)
        
        while True:
            changed = False
            for edge in list(all_edges):
                changed |= add_edge(edge)
                position, t1, t2 = edge.position, edge.tile1, edge.tile2
                new_edges = []
                if edge.position == 'left':
                    new_edges.append(('left', edge_completions[(t1, 'above')],  edge_completions[(t2, 'above')]))
                    new_edges.append(('left', edge_completions[('above', t1)], edge_completions[('above', t2)]))
                elif edge.position == 'above':
                    new_edges.append(('above', edge_completions[(t1, 'left')],  edge_completions[(t2, 'left')]))
                    new_edges.append(('above', edge_completions[('left', t1)], edge_completions[('left', t2)]))
                for position, t1, t2 in new_edges:
                    if t1 and t2:
                        changed |= add_edge(Edge(position, t1, t2))
            if not changed: break
        return all_edges, edge_completions

    def is_solved(self, m, n):
        all_edges, _ = self.complete_edges()
        return len(all_edges) == (m * (m - 1) + n * (n - 1))

    def corners(self):
        all_edges, edge_completions = self.complete_edges()
        tl = tr = bl = br = None
        for t in self.tiles:
            if not edge_completions[('above', t)] and not edge_completions[('left', t)]:
                tl = t
            elif not edge_completions[('above', t)] and not edge_completions[(t, 'left')]:
                tr = t
            elif not edge_completions[(t, 'above')] and not edge_completions[('left', t)]:
                bl = t
            elif not edge_completions[(t, 'above')] and not edge_completions[(t, 'left')]:
                br = t
        return tl, tr, bl, br

def place_all(tiles, size):
    possibilities = []
    def add_possibility(tile_map, remaining):
        heapq.heappush(possibilities, (len(remaining), tile_map, frozenset(remaining)))

    initial_tile_map = list(TileMap().try_placing(tiles[0]))[0]
    add_possibility(initial_tile_map, tiles[1:])

    while possibilities:
        _, tile_map, remaining = heapq.heappop(possibilities)
        if not remaining and tile_map.is_solved(size, size): return tile_map
        for tile in remaining:
            new_remaining = remaining - {tile}
            for new_tile_map in tile_map.try_placing(tile):
                add_possibility(new_tile_map, new_remaining)

In [408]:
test_input = """
Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###..."""

test_tiles = [Tile.from_record(record) for record in test_input.strip().split('\n\n')]
tiles_by_id = {t.tile_id: t for t in test_tiles}

In [410]:
solution = place_all(test_tiles, 3)
for e in solution.edges:
    print(e.position, e.tile1.tile_id, e.tile2.tile_id)

above 2971 2729
left 2729 1427
above 1489 1427
above 1427 2311
left 1951 2311
left 1427 2473
left 2311 3079
left 1489 1171


In [411]:
final_edges, _ = solution.complete_edges()

In [412]:
for e in final_edges:
    print(e.position, e.tile1.tile_id, e.tile2.tile_id)

above 2729 1951
above 1427 2311
left 1427 2473
left 2311 3079
left 2971 1489
above 1171 2473
above 2971 2729
left 2729 1427
above 1489 1427
above 2473 3079
left 1951 2311
left 1489 1171


In [413]:
product(t.tile_id for t in solution.corners())

20899048083289

In [396]:
def neighboring_cells(cell):
    x, y = cell
    return ((x - 1, y), (x, y - 1), (x + 1, y), (x, y + 1))

class TileMapV2:
    def __init__(self, tiles=None, cells=None):
        self.tiles = tiles or frozenset()
        self.cells = cells or {}

    def available_cells(self):
        if not self.cells:
            yield (0, 0)
        else:
            for cell in self.cells:
                for neighbor in neighboring_cells(cell):
                    if not self.cells.get(neighbor):
                        yield neighbor
            
    def neighboring_tiles(self, cell):
        return [self.cells.get(neighbor) for neighbor in neighboring_cells(cell)]

    def fits(self, tile, cell):
        neighbors = self.neighboring_tiles(cell)
        left_tile, above_tile, right_tile, below_tile = neighbors
        return ((not left_tile or left_tile.fits_left_of(tile)) and
                (not right_tile or tile.fits_left_of(right_tile)) and
                (not above_tile or above_tile.fits_above(tile)) and
                (not below_tile or tile.fits_above(below_tile)))

    def place(self, tile, cell):
        new_cells = dict(self.cells)
        new_cells[cell] = tile
        return TileMapV2(self.tiles | {tile}, new_cells)

    def bounds(self):
        x_lo = min(self.cells, key=lambda xy: xy[0])[0]
        x_hi = max(self.cells, key=lambda xy: xy[0])[0]
        y_lo = min(self.cells, key=lambda xy: xy[1])[1]
        y_hi = max(self.cells, key=lambda xy: xy[1])[1]
        return x_lo, x_hi, y_lo, y_hi

    def size(self):
        x_lo, x_hi, y_lo, y_hi = self.bounds()
        return x_hi - x_lo + 1, y_hi - y_lo + 1

    def is_filled(self):
        x_lo, x_hi, y_lo, y_hi = self.bounds()
        for x in range(x_lo, x_hi+1):
            for y in range(y_lo, y_hi+1):
                if not self.cells.get((x, y)):
                    return False
        return True

    def corners(self):
        x_lo, x_hi, y_lo, y_hi = self.bounds()
        return [self.cells[(x_lo, y_lo)],
                self.cells[(x_hi, y_lo)],
                self.cells[(x_lo, y_hi)],
                self.cells[(x_hi, y_hi)]]

    def print(self):
        x_lo, x_hi, y_lo, y_hi = self.bounds()
        for y in range(y_lo, y_hi+1):
            for x in range(x_lo, x_hi+1):
                print(f'{self.cells[(x,y)].tile_id if (x,y) in self.cells else ".":<6}', end='')
            print()

    def __lt__(self, other):
        return len(self.tiles) < len(other.tiles)


def place_one(tile_map, tile):
    orientations = list(tile.orientations()) if tile_map.tiles else [tile]
    for cell in tile_map.available_cells():
        for tile in orientations:
            if tile_map.fits(tile, cell):
                yield cell, tile_map.place(tile, cell)


def place_all(tiles, size):
    possibilities = []
    def add_possibility(tile_map, remaining, cell):
        p1 = len(remaining)
        p2 = -sum(t is not None for t in tile_map.neighboring_tiles(cell))
        heapq.heappush(possibilities, ((p1, p2), tile_map, remaining))

    def get_possibility():
        return heapq.heappop(possibilities)[1:]

    for cell, tile_map in place_one(TileMapV2(), tiles[0]):
        add_possibility(tile_map, frozenset(tiles[1:]), cell)

    while possibilities:
        tile_map, remaining = get_possibility()
        if not remaining and tile_map.is_filled(): return tile_map
        for tile in remaining:
            new_remaining = remaining - {tile}
            for cell, new_tile_map in place_one(tile_map, tile):
                if all(dim <= size for dim in new_tile_map.size()):
                    add_possibility(new_tile_map, new_remaining, cell)

In [397]:
tile_map = TileMapV2()
for tile_id in [1951, 2311, 3079, 2729, 1427]:
    print('placing', tile_id)
    for cell, tile_map in place_one(tile_map, tiles_by_id[tile_id]):
        tile_map.print()
        print()

placing 1951
1951  

placing 2311
1951  2311  

placing 3079
1951  2311  3079  

placing 2729
2729  .     .     
1951  2311  3079  

placing 1427
2729  1427  .     
1951  2311  3079  

2729  1427  .     
1951  2311  3079  



In [394]:
tile_map = place_all(test_tiles, 3)
tile_map.print()

2971  1489  1171  
2729  1427  2473  
1951  2311  3079  


In [395]:
%%time
# Final answers
with open('day20.txt') as f:
    tiles = [Tile.from_record(record) for record in f.read().strip().split('\n\n')]
    solution = place_all(tiles, 12)
    print('Part 1: ', product(t.tile_id for t in solution.corners()))

Part 1:  16192267830719
Wall time: 6.09 s
